In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tensorflow as tf
from numpy import *
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras import layers,datasets
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from tensorflow import keras
import gc
# import warnings
# warnings.filterwarnings('ignore')
import lightgbm as lgb
import category_encoders as ce
from sklearn.metrics import f1_score,classification_report,roc_auc_score
from impala.dbapi import connect
from impala.util import as_pandas
import pymysql

In [2]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,VarLenSparseFeat,get_feature_names

In [3]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [4]:
def get_game_id():
    conn = pymysql.connect(host='192.168.0.79', port=3306, user='readonly',
                           passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL 
    '''
    cur.execute(sql)
    result_df = cur.fetchall()
    cur.close()
    conn.close()
    return result_df

DeepCTR version 0.8.5 detected. Your version is 0.8.0.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.5


In [5]:
# 获取近期所有计划('2021年3月14号开始')
def get_plan_info():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='192.168.0.65', port=3306, user='readonly',
                           passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
                * 
            FROM
                db_ptom.ptom_third_plan p
            WHERE
                game_id IN ({})
                AND media_id = 10
                AND create_time>='2021-03-14'
                AND create_time<= date(NOW())
                            AND plan_id >= (
                                select plan_id from db_ptom.ptom_plan
                                where create_time >= '2021-03-14'
                                and create_time <= '2021-03-15'
                                limit 1
                            )
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result_df

In [6]:
# 获取创意数据('2021年1月1号开始')
def get_creative():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='192.168.0.65', port=3306, user='readonly',
                           passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
    SELECT
        a.plan_name,
        a.chl_user_id as channel_id,
        a.source_id,
        b.creative_param->'$.third_industry_id' as 'third_industry_id',
        b.creative_param->'$.ad_keywords' as 'ad_keywords',
        b.creative_param->'$.title_list' as 'title_list'
        FROM
        db_ptom.ptom_plan as a
        LEFT JOIN
        db_ptom.ptom_batch_ad_task as b
        on a.plan_name = b.plan_name
        WHERE
          a.media_id = 10
            AND b.media_id = 10
            and a.game_id IN ({})
            AND a.create_time>='2021-03-14'
            AND b.create_time>='2021-03-14'
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result_df

In [7]:
# 获取image_id,label_ids
def get_image_info():
    conn = pymysql.connect(host='192.168.0.79', port=3306, user='readonly',
                        passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id,
            a.image_id,
            b.label_ids
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN
            db_data_ptom.ptom_image_info b
        on a.image_id = b.image_id
        WHERE
            a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
            AND a.media_id = 10 
            AND a.create_time >= '2021-03-14' 
        GROUP BY
            a.chl_user_id,
            a.source_id,
            a.image_id
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    return result_df

In [8]:
# 获取计划运营指标
def get_launch_report():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='192.168.0.79', port=3306, user='readonly',
                           passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id AS source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount as roi,
            b.pay_role_user_num / b.create_role_num as pay_rate
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN db_stdata.st_lauch_report b ON a.chl_user_id = b.channel_id 
            AND a.source_id = b.source_id 
        WHERE
            a.create_time >= '2021-03-15' 
            AND b.tdate >= '2021-03-15' 
            AND b.tdate_type = 'day' 
            AND b.media_id = 10
            AND b.game_id IN ({})
            AND b.amount >= 500
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')

    return result_df

In [9]:
# 获取近期账号维度窗口期统计数据
def getAdData(conn, begin, end, n):
    originSql = '''
              SELECT
                a.ad_account_id AS 'ad_account_id',
                ifnull( a.ad_amount, 0 ) AS 'ad_amount',
                ifnull( a.ad_create_role_num, 0 ) AS 'ad_create_role_num',
                ifnull( a.ad_new_role_money, 0 ) AS 'ad_new_role_money',
                ifnull( a.ad_pay_role_user_num, 0 ) AS 'ad_pay_role_user_num',
                ifnull( a.ad_source_num, 0 ) AS 'ad_source_num',
                IFNULL( b.ad_valid_source_num, 0 ) AS 'ad_valid_source_num',
                (
                CASE
                        
                        WHEN ifnull( a.ad_create_role_num, 0 )= 0 THEN
                        0 ELSE IFNULL( a.ad_pay_role_user_num, 0 ) / ifnull( a.ad_create_role_num, 0 ) 
                    END 
                    ) AS 'ad_pay_rate',
                    (
                    CASE
                            
                            WHEN ifnull( a.ad_create_role_num, 0 )= 0 THEN
                            0 ELSE IFNULL( a.ad_amount, 0 ) / ifnull( a.ad_create_role_num, 0 ) 
                        END 
                        ) AS 'ad_create_role_cost',
                        (
                        CASE
                                
                                WHEN ifnull( a.ad_pay_role_user_num, 0 )= 0 THEN
                                0 ELSE IFNULL( a.ad_amount, 0 ) / ifnull( a.ad_pay_role_user_num, 0 ) 
                            END 
                            ) AS 'ad_create_role_pay_cost',
                            (
                            CASE
                                    
                                    WHEN ifnull( a.ad_source_num, 0 )= 0 THEN
                                    0 ELSE IFNULL( b.ad_valid_source_num, 0 ) / ifnull( a.ad_source_num, 0 ) 
                                END 
                                ) AS 'ad_valid_source_rate',
                                (
                                CASE
                                        
                                        WHEN ifnull( a.ad_amount, 0 )= 0 THEN
                                        0 ELSE IFNULL( a.ad_new_role_money, 0 ) / ifnull( a.ad_amount, 0 ) 
                                    END 
                                    ) AS 'ad_create_role_roi' 
                                FROM
                                    (
                                    SELECT
                                        a.ad_account_id,
                                        ifnull( sum( a.amount ), 0 ) AS 'ad_amount',
                                        ifnull( sum( a.create_role_num ), 0 ) AS 'ad_create_role_num',
                                        ifnull( sum( b.new_role_money ), 0 ) AS 'ad_new_role_money',
                                        ifnull( sum( b.pay_role_user_num ), 0 ) AS 'ad_pay_role_user_num',
                                        count( DISTINCT a.source_id ) AS 'ad_source_num' 
                                    FROM
                                        (
                                        SELECT
                                            b.ad_account_id,
                                            b.doc_id,
                                            a.game_id,
                                            a.channel_id,
                                            a.source_id,
                                            a.amount,
                                            a.create_role_num 
                                        FROM
                                            db_stdata.st_lauch_report a
                                            INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                            AND a.source_id = b.source_id 
                                            AND a.channel_id = b.chl_user_id 
                                        WHERE
                                            a.tdate_type = 'day' 
                                            AND a.tdate >= '{begin}' 
                                            AND a.tdate <= '{end}' AND a.amount > 100 
                                            AND a.media_id = 10 
                                        AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL )) a
                                        LEFT JOIN (
                                        SELECT
                                            c.game_id,
                                            c.channel_id,
                                            c.source_id,
                                            b.ad_account_id,
                                            sum( c.create_role_money ) new_role_money,
                                            IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                        FROM
                                            db_stdata.st_game_days c
                                            INNER JOIN db_data_ptom.ptom_plan b ON c.game_id = b.game_id 
                                            AND c.source_id = b.source_id 
                                            AND c.channel_id = b.chl_user_id 
                                        WHERE
                                            c.report_days = {n} 
                                            AND c.tdate = '{end}' 
                                            AND c.tdate_type = 'day' 
                                            AND c.query_type = 13 
                                            AND c.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                                        GROUP BY
                                            c.game_id,
                                            c.channel_id,
                                            c.source_id 
                                        HAVING
                                            ( new_role_money > 0 OR pay_role_user_num > 0 ) 
                                        ) b ON a.source_id = b.source_id 
                                        AND a.channel_id = b.channel_id 
                                    GROUP BY
                                        a.ad_account_id 
                                    ) a
                                    LEFT JOIN (
                                    SELECT
                                        b.ad_account_id,
                                        count( DISTINCT b.plan_id ) AS 'ad_valid_source_num' 
                                    FROM
                                        db_stdata.st_lauch_report a
                                        INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                        AND a.source_id = b.source_id 
                                        AND a.channel_id = b.chl_user_id 
                                    WHERE
                                        a.tdate_type = 'day' 
                                        AND a.tdate >= '{begin}' 
                                        AND a.tdate <= '{end}' AND a.amount > 100  
                                        AND a.media_id = 10 
                                        AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                                        AND a.pay_role_user_num > 0 
                                    GROUP BY
                                    b.ad_account_id 
                ) b ON A.ad_account_id = B.ad_account_id
    '''
    finalSql = originSql.format(begin=begin, end=end, n=n)
    result = pd.read_sql(finalSql, conn)
    return result

In [10]:
# 获取素材报表数据
def getImData(conn, begin, end, n):
    originSql = '''
            SELECT
                a.image_id AS 'image_id',
                ifnull( a.im_amount, 0 ) AS 'im_amount',
                ifnull( a.im_create_role_num, 0 ) AS 'im_create_role_num',
                ifnull( a.im_new_role_money, 0 ) AS 'im_new_role_money',
                ifnull( a.im_pay_role_user_num, 0 ) AS 'im_pay_role_user_num',
                ifnull( a.im_source_num, 0 ) AS 'im_source_num',
                IFNULL( b.im_valid_source_num, 0 ) AS 'im_valid_source_num',
                (
                CASE
                        
                        WHEN ifnull( a.im_create_role_num, 0 )= 0 THEN
                        0 ELSE IFNULL( a.im_pay_role_user_num, 0 ) / ifnull( a.im_create_role_num, 0 ) 
                    END 
                    ) AS 'im_pay_rate',
                    (
                    CASE
                            
                            WHEN ifnull( a.im_create_role_num, 0 )= 0 THEN
                            0 ELSE IFNULL( a.im_amount, 0 ) / ifnull( a.im_create_role_num, 0 ) 
                        END 
                        ) AS 'im_create_role_cost',
                        (
                        CASE
                                
                                WHEN ifnull( a.im_pay_role_user_num, 0 )= 0 THEN
                                0 ELSE IFNULL( a.im_amount, 0 ) / ifnull( a.im_pay_role_user_num, 0 ) 
                            END 
                            ) AS 'im_create_role_pay_cost',
                            (
                            CASE
                                    
                                    WHEN ifnull( a.im_source_num, 0 )= 0 THEN
                                    0 ELSE IFNULL( b.im_valid_source_num, 0 ) / ifnull( a.im_source_num, 0 ) 
                                END 
                                ) AS 'im_valid_source_rate',
                                (
                                CASE
                                        
                                        WHEN ifnull( a.im_amount, 0 )= 0 THEN
                                        0 ELSE IFNULL( a.im_new_role_money, 0 ) / ifnull( a.im_amount, 0 ) 
                                    END 
                                    ) AS 'image_create_role_roi' 
                                FROM
                                    (
                                    SELECT
                                        a.image_id,
                                        ifnull( sum( a.amount ), 0 ) AS 'im_amount',
                                        ifnull( sum( a.create_role_num ), 0 ) AS 'im_create_role_num',
                                        ifnull( sum( b.new_role_money ), 0 ) AS 'im_new_role_money',
                                        ifnull( sum( b.pay_role_user_num ), 0 ) AS 'im_pay_role_user_num',
                                        count( DISTINCT a.source_id ) AS 'im_source_num' 
                                    FROM
                                        (
                                        SELECT
                                            b.image_id,
                                            a.game_id,
                                            a.channel_id,
                                            a.source_id,
                                            a.amount,
                                            a.create_role_num 
                                        FROM
                                            db_stdata.st_lauch_report a
                                            INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                            AND a.source_id = b.source_id 
                                            AND a.channel_id = b.chl_user_id                                        
                                        WHERE
                                            a.tdate_type = 'day' 
                                            AND a.tdate >= '{begin}' 
                                            AND a.tdate <= '{end}' AND a.amount > 100 
                                            AND b.image_id IS NOT NULL 
                                            AND b.image_id <> '' 
                                            AND a.media_id = 10 
                                        AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL )) a
                                        LEFT JOIN (
                                        SELECT
                                            c.game_id,
                                            c.channel_id,
                                            c.source_id,
                                            b.image_id,
                                            sum( c.create_role_money ) new_role_money,
                                            IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                                        FROM
                                            db_stdata.st_game_days c
                                            INNER JOIN db_data_ptom.ptom_plan b ON c.game_id = b.game_id 
                                            AND c.source_id = b.source_id 
                                            AND c.channel_id = b.chl_user_id 
                                        WHERE
                                            c.report_days = {n} 
                                            AND c.tdate = '{end}' 
                                            AND c.tdate_type = 'day' 
                                            AND c.query_type = 13 
                                            AND b.image_id IS NOT NULL 
                                            AND b.image_id <> '' 
                                            AND c.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                                        GROUP BY
                                            c.game_id,
                                            c.channel_id,
                                            c.source_id 
                                        HAVING
                                            ( new_role_money > 0 OR pay_role_user_num > 0 ) 
                                        ) b ON a.source_id = b.source_id 
                                        AND a.channel_id = b.channel_id 
                                    GROUP BY
                                        a.image_id 
                                    ) a
                                    LEFT JOIN (
                                    SELECT
                                        b.image_id,
                                        count( DISTINCT b.plan_id ) AS 'im_valid_source_num' 
                                    FROM
                                        db_stdata.st_lauch_report a
                                        INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                        AND a.source_id = b.source_id 
                                        AND a.channel_id = b.chl_user_id 
                                    WHERE
                                        a.tdate_type = 'day' 
                                        AND a.tdate >= '{begin}' 
                                        AND a.tdate <= '{end}' AND a.amount > 100 
                                        AND b.image_id IS NOT NULL 
                                        AND b.image_id <> '' 
                                        AND a.media_id = 10 
                                        AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                                        AND a.pay_role_user_num > 0 
                                    GROUP BY
                                        b.image_id 
                                    ) b ON A.image_id = B.image_id
    '''
    finalSql = originSql.format(begin=begin, end=end, n=n)
    result = pd.read_sql(finalSql, conn)
    return result

In [11]:
def etl_data(start, end, n):
    '''
    获取账号、素材、账号+素材维度的数据
    :return:
    '''
    # 链接数据库，并创建游标
    conn1 = pymysql.connect(host='192.168.0.79', port=3306, user='readonly',
                            passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t', db='db_data')
#     columns = ['ad_account_id', 'ad_amount', 'ad_create_role_num', 'ad_new_role_money',
#                'ad_pay_role_user_num', 'ad_source_num', 'ad_valid_source_num',
#                'ad_pay_rate', 'ad_create_role_cost', 'ad_create_role_pay_cost',
#                'ad_valid_source_rate', 'ad_create_role_roi', 'model_run_datetime']
    result_im = pd.DataFrame()
    result_ad = pd.DataFrame()
#     result_ad_im = pd.DataFrame()
    date_list = pd.date_range(start=start, end=end)
    for date in date_list:
        end = date
        begin = date - pd.Timedelta(days=n - 1)
        end = str(end).split(' ')[0]
        begin = str(begin).split(' ')[0]
        # 获取素材窗口期数据
        ImData = getImData(conn1, begin, end, n=n)
        ImData['model_run_datetime'] = date + pd.Timedelta(days=1)
        # 获取账号窗口期数据
        AdData = getAdData(conn1, begin, end, n=n)
        AdData['model_run_datetime'] = date + pd.Timedelta(days=1)
#         # 获取账号+素材窗口期数据
#         AdImData = getAdImData(conn1, begin, end, n=n)
#         AdImData['model_run_datetime'] = date + pd.Timedelta(days=1)
        result_im = result_im.append(ImData)
        result_ad = result_ad.append(AdData)
#         result_ad_im = result_ad_im.append(AdImData)

    conn1.close()
    return result_im, result_ad

In [12]:
# 获取账号、素材、账号+素材维度的数据
# 读取近期数据
for n in (3, 5):
    train_im,train_ad = etl_data(start='2021-03-13', end=str((pd.datetime.now() - pd.DateOffset(1)).date()), n=n)
    train_im.columns = [i + '_' + str(n) for i in train_im.columns]
    train_ad.columns = [i + '_' + str(n) for i in train_ad.columns]
#     train_ad_im.columns = [i + '_' + str(n) for i in train_ad_im.columns]

    train_im.rename(columns={'model_run_datetime_' + str(n): 'create_date',
                          'image_id_' + str(n): 'image_id'}, inplace=True)
    train_ad.rename(columns={'model_run_datetime_' + str(n): 'create_date',
                          'ad_account_id_' + str(n): 'ad_account_id'}, inplace=True)
#     train_ad_im.rename(columns={'model_run_datetime_' + str(n): 'create_date',
#                               'ad_account_id_' + str(n): 'ad_account_id', 
#                               'image_id_' + str(n): 'image_id'}, inplace=True)
    exec("train_im_%d = train_im"%(n))
    exec("train_ad_%d = train_ad"%(n))
#     exec("train_ad_im_%d = train_ad_im"%(n))
# 读取历史数据
# train_im_data_1 = pd.read_csv('./hist_data/im_data/train_data_1.csv')
train_im_data_3 = pd.read_csv('./hist_data/im_data/train_data_3.csv')
train_im_data_5 = pd.read_csv('./hist_data/im_data/train_data_5.csv')
# train_im_data_7 = pd.read_csv('./hist_data/im_data/train_data_7.csv')
# train_ad_data_1 = pd.read_csv('./hist_data/ad_data/train_data_1.csv')
train_ad_data_3 = pd.read_csv('./hist_data/ad_data/train_data_3.csv')
train_ad_data_5 = pd.read_csv('./hist_data/ad_data/train_data_5.csv')
# train_ad_data_7 = pd.read_csv('./hist_data/ad_data/train_data_7.csv')
# train_ad_im_data_1 = pd.read_csv('./hist_data/ad_im_data/train_data_1.csv')
# train_ad_im_data_3 = pd.read_csv('./hist_data/ad_im_data/train_data_3.csv')
# train_ad_im_data_5 = pd.read_csv('./hist_data/ad_im_data/train_data_5.csv')
# train_ad_im_data_7 = pd.read_csv('./hist_data/ad_im_data/train_data_7.csv')    
# 合并数据
# train_im_data_1 = train_im_data_1.append(train_im_1)
train_im_data_3 = train_im_data_3.append(train_im_3)
train_im_data_5 = train_im_data_5.append(train_im_5)
# train_im_data_7 = train_im_data_7.append(train_im_7)
# train_ad_data_1 = train_ad_data_1.append(train_ad_1)
train_ad_data_3 = train_ad_data_3.append(train_ad_3)
train_ad_data_5 = train_ad_data_5.append(train_ad_5)
# train_ad_data_7 = train_ad_data_7.append(train_ad_7)
# train_ad_im_data_1 = train_ad_im_data_1.append(train_ad_im_1)
# train_ad_im_data_3 = train_ad_im_data_3.append(train_ad_im_3)
# train_ad_im_data_5 = train_ad_im_data_5.append(train_ad_im_5)
# train_ad_im_data_7 = train_ad_im_data_7.append(train_ad_im_7)

for i in [3,5]:
    exec("train_im_data_%d['image_id'] = train_im_data_%d['image_id'].astype('int')"%(i,i))
    exec("train_im_data_%d['create_date'] = pd.to_datetime(train_im_data_%d['create_date'])"%(i,i))
    exec("train_ad_data_%d['ad_account_id'] = train_ad_data_%d['ad_account_id'].astype('int')"%(i,i))
    exec("train_ad_data_%d['create_date'] = pd.to_datetime(train_ad_data_%d['create_date'])"%(i,i))
#     exec("train_ad_im_data_%d['image_id'] = train_ad_im_data_%d['image_id'].astype('int')"%(i,i))
#     exec("train_ad_im_data_%d['ad_account_id'] = train_ad_im_data_%d['ad_account_id'].astype('int')"%(i,i))
#     exec("train_ad_im_data_%d['create_date'] = pd.to_datetime(train_ad_im_data_%d['create_date'])"%(i,i))
    
def fill_date(df,col):
    result_df = pd.DataFrame()
    for im_id in df[col].unique():
        temp = df[df[col]==im_id]
        im_date = temp['create_date'].values
        all_date = pd.date_range(temp['create_date'].min()- pd.DateOffset(days=1),temp['create_date'].max(),freq='D')
        new_date = pd.DataFrame(np.setdiff1d(all_date,im_date),columns=['create_date'])
        new_date[col]=im_id
        temp = pd.concat([temp,new_date])
        temp.sort_values(by='create_date',inplace=True)
        temp = temp.reset_index(drop=True)
        temp = temp.fillna(method='ffill')
        temp = temp.fillna(method='bfill')
        result_df = result_df.append(temp)
    return result_df

# train_im_data_1 = fill_date(train_im_data_1,col='image_id')
train_im_data_3 = fill_date(train_im_data_3,col='image_id')
train_im_data_5 = fill_date(train_im_data_5,col='image_id')
# train_im_data_7 = fill_date(train_im_data_7,col='image_id')
# train_ad_data_1 = fill_date(train_ad_data_1,col='ad_account_id')
train_ad_data_3 = fill_date(train_ad_data_3,col='ad_account_id')
train_ad_data_5 = fill_date(train_ad_data_5,col='ad_account_id')
# train_ad_data_7 = fill_date(train_ad_data_7,col='ad_account_id')

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


In [13]:
# 解析json
def get_plan_json(plan_info):
    plan_info.drop(['inventory_type','budget'], axis=1, inplace=True)
    plan_info.dropna(how='all', inplace=True, axis=1)
    plan_info.dropna(subset=['ad_info'],inplace=True)
    # 解析json
    plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
    temp = plan_info['ad_info'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('ad_info', axis=1, inplace=True)
    temp = plan_info['audience'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('audience', axis=1, inplace=True)
    temp = plan_info['action'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('action', axis=1, inplace=True)
    plan_info.dropna(how='all', inplace=True, axis=1)
    plan_info = plan_info[['ad_account_id', 'game_id', 'channel_id', 'source_id',
           'create_time', 'smart_bid_type', 'hide_if_exists', 'budget',
           'delivery_range', 'adjust_cpa', 'inventory_type', 'hide_if_converted',
           'flow_control_mode', 'schedule_time', 'cpa_bid', 'auto_extend_enabled',
           'gender', 'city', 'platform', 'launch_price',
           'retargeting_tags_exclude', 'interest_categories',
           'ac', 'android_osv', 'location_type', 'retargeting_tags_include',
           'retargeting_type', 'ios_osv', 'interest_action_mode', 'age',
           'action_categories', 'action_days', 'action_scene','deep_bid_type','roi_goal']]
    return plan_info

In [14]:
def get_all_data():
    # 读取历史数据
    plan_info = pd.read_csv('./hist_data/ptom_third_plan.csv')
    image_info = pd.read_csv('./hist_data/image_info.csv')
    launch_report = pd.read_csv('./hist_data/launch_report.csv')
    
    plan_info_new = get_plan_info()
    image_info_new = get_image_info()
    launch_report_new = get_launch_report()
    
    plan_info = plan_info.append(plan_info_new)
    plan_info = get_plan_json(plan_info)
    image_info = image_info.append(image_info_new)
    launch_report = launch_report.append(launch_report_new)
    image_info.dropna(subset=['image_id'],inplace=True)
    image_info['image_id'] = image_info['image_id'].astype(int)
    return plan_info,image_info,launch_report

In [15]:
plan_info,image_info,launch_report = get_all_data()

In [15]:
## 建计划

In [16]:
# 获取score_image (分数大于550的image_id)
def get_score_image():
    conn = connect(host='192.168.0.88', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = 'select image_id,label_ids from dws.dws_image_score_d where media_id=10 and score>=580 and dt=CURRENT_DATE group by image_id,label_ids'
    cursor.execute(sql_engine)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result['label_ids'] = result['label_ids'].astype(str)
    result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
    result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
    result = result[result['label_ids'].isin([19,25,81,103,112,136,152,20,36,41,78,81,98,119,126,27,60,93,102,103])]
#     print(result)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result['image_id'].values

In [17]:
plan_info['create_time'] = pd.to_datetime(plan_info['create_time'])
plan_info_current = plan_info[plan_info['create_time']>=pd.datetime.now() - pd.DateOffset(8)]

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [18]:
# 获取近期计划的运营数据
def get_now_plan_roi():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='192.168.0.79', port=3306, user='readonly',
                           passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
          b.channel_id,
          b.source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount AS roi,
            b.pay_role_user_num / b.create_role_num AS pay_rate 
        FROM
            db_stdata.st_lauch_report b
        WHERE
            b.tdate >= date( NOW() - INTERVAL 120 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 10 
            AND b.game_id IN ({}) 
            AND b.amount >= 500 
            AND b.pay_role_user_num >= 2 
            AND b.new_role_money >= 128
            AND (b.new_role_money / b.amount)>=0.03
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')
#     result_df = result_df[result_df['roi'] >= 0.03]
    return result_df

In [19]:
now_plan_roi = get_now_plan_roi()

In [20]:
score_image = get_score_image()

In [21]:
df_create = pd.merge(plan_info_current,image_info,on=['channel_id','source_id'],how='left')
df_create = pd.merge(df_create,now_plan_roi,on=['channel_id','source_id'],how='inner')
df_create['platform'] = df_create['platform'].astype(str)
df_create['platform'] = df_create['platform'].map({"['ANDROID']":1,"['IOS']":2})
df_create['platform'] = df_create['platform'].astype(int)
df_create.dropna(subset=['image_id'],inplace=True)
df_create['image_id'] = df_create['image_id'].astype(int)

In [22]:
def create_plan(df,score_image):
    # 确定game_id 
    game_id = 1001379
    df = df[df['game_id']==game_id]
    
    # 选ad_account_id、image_id每个账号+素材8条
    ad_account_id_group = np.array([6620,6621,6867,6868])
    image_id_group =np.intersect1d(df['image_id'].unique(),score_image)
    plan = pd.DataFrame()
    for ad_account in ad_account_id_group:
        for image in image_id_group:
    #         print(image)
            temp = pd.DataFrame({'ad_account_id':[ad_account],'image_id':[image]})
            plan = plan.append(temp)
    #         print(temp)
    plan = pd.DataFrame(np.repeat(plan.values,8,axis=0), columns=plan.columns)
    
    plan['game_id'] = game_id
    

    # 选platform测试期默认[ANDROID]
    plan['platform']= df['platform'].iloc[0]
    plan['platform']= plan['platform'].apply(lambda x:'[ANDROID]' if x==1 else '[IOS]')

    # 选android_osv
    count_df = pd.DataFrame(data=df['android_osv'].value_counts()).reset_index()
    count_df.columns = ['col','counts']
    count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
    plan['android_osv']=plan['platform'].apply(lambda x:'NONE' if x=='[IOS]' else 
                                               np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0])

    # 选ios_osv
    count_df = pd.DataFrame(data=df['ios_osv'].value_counts()).reset_index()
    count_df.columns = ['col','counts']
    count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
    plan['ios_osv']=plan['platform'].apply(lambda x:'NONE' if x=='[ANDROID]' else 
                                               np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0])

    # 选budget 
    plan['budget']= plan['platform'].apply(lambda x:3300 if x=='[ANDROID]' else 4000)

    # 选'retargeting_type','retargeting_tags_include','retargeting_tags_exclude'  人群包定向
    sample_df = df[['retargeting_type', 'retargeting_tags_include', 'retargeting_tags_exclude']]
    sample_df = sample_df.sample(n=plan.shape[0], replace=True).reset_index(drop=True)
    plan = pd.concat([plan, sample_df], axis=1)

    # 选'interest_action_mode','action_scene','action_days','action_categories' ,'interest_categories' 行为兴趣
    sample_df = df[['interest_action_mode','action_scene','action_days','action_categories','interest_categories']]
    sample_df = sample_df.sample(n=plan.shape[0], replace=True).reset_index(drop=True)
    plan = pd.concat([plan, sample_df], axis=1)

    # 选'deep_bid_type','roi_goal','smart_bid_type','adjust_cpa','cpa_bid'出价方式
    sample_df = df[['deep_bid_type','roi_goal','smart_bid_type','adjust_cpa','cpa_bid']]
    sample_df = sample_df[sample_df['deep_bid_type'].isin(['ROI_COEFFICIENT','BID_PER_ACTION'])]
    sample_df = sample_df.sample(n=plan.shape[0], replace=True).reset_index(drop=True)
    plan = pd.concat([plan, sample_df], axis=1)

    # 选
    cols = ['delivery_range','city','location_type','gender','age','ac','launch_price',
           'auto_extend_enabled','hide_if_exists','hide_if_converted','schedule_time','flow_control_mode']
    for col in cols:
        count_df = pd.DataFrame(data=df[col].value_counts()).reset_index()
        count_df.columns = ['col','counts']
        count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
        plan[col]=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)

    # 选inventory_type 创意投放位置
    count_df = pd.DataFrame(data=df['inventory_type'].value_counts()).reset_index()
    count_df.columns = ['col','counts']

    for i in range(count_df.shape[0]):
        values = count_df.loc[i,'col']

        for value in values:
            if value== None:
                values.remove(value)
        a = np.empty(1, dtype=object)
        a[0] = values
        count_df.loc[i,'col'] = a
    count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
    plan['inventory_type']=plan.apply(lambda x:np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0], axis=1)
    plan['create_time'] = pd.to_datetime(pd.datetime.now())
    plan['create_date'] = pd.to_datetime(pd.datetime.now().date())
    
    return plan

In [23]:
plan_create = create_plan(df_create, score_image)

C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:82: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:83: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.


In [24]:
image_info.dropna(subset=['image_id'],inplace=True)
image_info['image_id'] = image_info['image_id'].astype(int)
plan_create = pd.merge(plan_create,image_info[['image_id','label_ids']].drop_duplicates(),on='image_id',how='left')

In [149]:
df = pd.merge(plan_info,image_info,on=['channel_id','source_id'],how='left')
df.dropna(subset=['image_id'],inplace=True)
df = pd.merge(df, launch_report,on=['channel_id','source_id'],how='left')
df.drop(df[df['tdate'].isna()].index,inplace=True)
df = df[df['amount']>=500]

df['platform'] = df['platform'].astype(str)
df['platform'] = df['platform'].map({"['ANDROID']":1,"['IOS']":2})
df['label'] = df.apply(lambda x: 1 if x.new_role_money/x.amount>=0.02 else 0,axis=1)
df['ad_account_id'] = df['ad_account_id'].astype('int')
df['image_id'] = df['image_id'].astype('int')
df.rename(columns={'tdate':'create_date'},inplace=True)
df['create_date'] = pd.to_datetime(df['create_date'])
df['create_time'] = pd.to_datetime(df['create_time'])
df.drop(['budget','cpa_bid','channel_id','source_id','amount','roi','pay_rate',
         'new_role_money'],axis=1,inplace=True)

In [150]:
plan_create_train = plan_create.drop(['budget', 'cpa_bid'],axis=1)
plan_create_train['platform'] = plan_create_train['platform'].map({'[ANDROID]':1,'[IOS]':2})
df['train_label'] = 1
plan_create_train['train_label'] = 0
plan_create_train['label'] = -1

In [151]:
df = df.append(plan_create_train)

In [152]:
df = pd.merge(df, train_im_data_3, on=['image_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, train_im_data_5, on=['image_id','create_date'],how='left', validate='many_to_one')

df = pd.merge(df, train_ad_data_3, on=['ad_account_id','create_date'],how='left', validate='many_to_one')
df = pd.merge(df, train_ad_data_5, on=['ad_account_id','create_date'],how='left', validate='many_to_one')

In [153]:
cost_col = ['im_create_role_cost_3',
            'im_create_role_cost_5',
            'ad_create_role_cost_3',
            'ad_create_role_cost_5',
          'im_create_role_pay_cost_3',
            'im_create_role_pay_cost_5',
            'ad_create_role_pay_cost_3',
            'ad_create_role_pay_cost_5'
           ]
for col in cost_col:
    df[col] = df[col].apply(lambda x:x if x>0 else np.inf)

In [154]:
df['create_date'] = pd.to_datetime(df['create_date'])
df['ad_im_sort_id'] = df.groupby(['ad_account_id','image_id'])['create_time'].rank()
df['ad_game_sort_id'] = df.groupby(['ad_account_id','game_id'])['create_time'].rank()
df['im_ad_sort_id'] = df.groupby(['image_id','ad_account_id'])['create_time'].rank()
df['image_rank'] = df.groupby(['image_id'])['create_time'].rank()

In [155]:
bucket_col = ['im_amount_3','im_pay_role_user_num_3','im_create_role_num_3','im_new_role_money_3','im_source_num_3','im_valid_source_num_3',
             'im_amount_5','im_pay_role_user_num_5','im_create_role_num_5','im_new_role_money_5','im_source_num_5','im_valid_source_num_5',
             'ad_amount_3','ad_pay_role_user_num_3','ad_create_role_num_3','ad_new_role_money_3','ad_source_num_3','ad_valid_source_num_3',
             'ad_amount_5','ad_pay_role_user_num_5','ad_create_role_num_5','ad_new_role_money_5','ad_source_num_5','ad_valid_source_num_5']

In [156]:
def optimal_binning_boundary(x: pd.Series, y: pd.Series, nan: float = -999.) -> list:
    '''
        利用决策树获得最优分箱的边界值列表
    '''
    boundary = []  # 待return的分箱边界值列表
    
    x = x.fillna(nan).values  # 填充缺失值
    y = y.values
    from sklearn.tree import DecisionTreeClassifier
    clf = DecisionTreeClassifier(criterion='entropy',    #“信息熵”最小化准则划分
                                 max_leaf_nodes=10,       # 最大叶子节点数
                                 min_samples_leaf=0.05)  # 叶子节点样本数量最小占比

    clf.fit(x.reshape(-1, 1), y)  # 训练决策树
    
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    threshold = clf.tree_.threshold
    
    for i in range(n_nodes):
        if children_left[i] != children_right[i]:  # 获得决策树节点上的划分边界值
            boundary.append(threshold[i])

    boundary.sort()

    min_x = x.min()
    max_x = x.max() + 0.1  # +0.1是为了考虑后续groupby操作时，能包含特征最大值的样本
    boundary = [min_x] + boundary + [max_x]

    return boundary

In [157]:
def bucketed(df, bucket_col):
    for col in bucket_col:
        df[col] = df[col].astype(np.float32)
        bins = optimal_binning_boundary(x=df[col],
                             y=df['label'])
        df[col] = pd.cut(df[col], bins, labels=range(1,len(bins)))
        df[col] = df[col].astype(np.float32)
        df[col] = df[col].fillna(0)
    return df

In [158]:
df = bucketed(df, bucket_col)

In [159]:

df['im_create_role_cost_3'] = pd.cut(df.im_create_role_cost_3, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['im_create_role_pay_cost_3'] = pd.cut(df.im_create_role_pay_cost_3, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])
df['im_create_role_cost_5'] = pd.cut(df.im_create_role_cost_5, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['im_create_role_pay_cost_5'] = pd.cut(df.im_create_role_pay_cost_5, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])

df['ad_create_role_cost_3'] = pd.cut(df.im_create_role_cost_3, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['ad_create_role_pay_cost_3'] = pd.cut(df.im_create_role_pay_cost_3, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])
df['ad_create_role_cost_5'] = pd.cut(df.im_create_role_cost_5, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['ad_create_role_pay_cost_5'] = pd.cut(df.im_create_role_pay_cost_5, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])

In [134]:
def get_mutil_feature(df):
    cols = ['inventory_type','age','city','retargeting_tags_include','retargeting_tags_exclude','ac','interest_categories',
       'action_scene','action_categories']
    for col in cols:
        df[col] = df[col].apply(lambda x:x if x==x else [])
        df[col] = df[col].apply(lambda x:list(filter(None, x)))
        df[col] = df[col].apply(lambda x:[str(i) for i in x])
        df[col] = df[col].str.join('|')
    return df

In [135]:
df = get_mutil_feature(df)

In [136]:
df.head()

,ad_account_id,game_id,create_time,smart_bid_type,hide_if_exists,delivery_range,adjust_cpa,inventory_type,hide_if_converted,flow_control_mode,schedule_time,auto_extend_enabled,gender,city,platform,launch_price,retargeting_tags_exclude,interest_categories,ac,android_osv,location_type,retargeting_tags_include,retargeting_type,ios_osv,interest_action_mode,age,action_categories,action_days,action_scene,deep_bid_type,roi_goal,image_id,label_ids,create_date,label,train_label,im_amount_3,im_create_role_num_3,im_new_role_money_3,im_pay_role_user_num_3,im_source_num_3,im_valid_source_num_3,im_pay_rate_3,im_create_role_cost_3,im_create_role_pay_cost_3,im_valid_source_rate_3,image_create_role_roi_3,im_amount_5,im_create_role_num_5,im_new_role_money_5,im_pay_role_user_num_5,im_source_num_5,im_valid_source_num_5,im_pay_rate_5,im_create_role_cost_5,im_create_role_pay_cost_5,im_valid_source_rate_5,image_create_role_roi_5,ad_amount_3,ad_create_role_num_3,ad_new_role_money_3,ad_pay_role_user_num_3,ad_source_num_3,ad_valid_source_num_3,ad_pay_rate_3,ad_create_role_cost_3,ad_create_role_pay_cost_3,ad_valid_source_rate_3,ad_create_role_roi_3,ad_amount_5,ad_create_role_num_5,ad_new_role_money_5,ad_pay_role_user_num_5,ad_source_num_5,ad_valid_source_num_5,ad_pay_rate_5,ad_create_role_cost_5,ad_create_role_pay_cost_5,ad_valid_source_rate_5,ad_create_role_roi_5,ad_im_sort_id,ad_game_sort_id,im_ad_sort_id,image_rank
0,4003,1000840,2020-01-19 16:18:31,SMART_BID_CONSERVATIVE,0,UNION,0.0,INVENTORY_UNION_SLOT,AD,FLOW_CONTROL_MODE_FAST,1111111111111111111111111111111111111111111111...,0,GENDER_MALE,,1,[],,,,NONE,CURRENT,,NONE,NONE,UNLIMITED,AGE_BETWEEN_24_30|AGE_BETWEEN_31_40|AGE_BETWEE...,,NaN,,NaN,NaN,19788,NaN,2020-01-21,0,1,6.0,6.0,4.0,6.0,5.0,3.0,0.0173,2,1,0.2222,0.047182,6.0,6.0,3.0,4.0,3.0,3.0,0.0173,2,1,0.2222,0.047182,6.0,7.0,4.0,4.0,2.0,3.0,0.0135,1,1,0.3333,0.005611,6.0,4.0,2.0,5.0,3.0,3.0,0.0135,1,1,0.3333,0.005611,1.0,1.0,1.0,1.0
1,4003,1000840,2020-01-20 14:35:23,SMART_BID_CONSERVATIVE,0,UNION,0.0,INVENTORY_UNION_SLOT,AD,FLOW_CONTROL_MODE_FAST,1111111111111111111111111111111111111111111111...,0,GENDER_MALE,,1,[],,,,NONE,CURRENT,,NONE,NONE,UNLIMITED,AGE_BETWEEN_24_30|AGE_BETWEEN_31_40|AGE_BETWEE...,,NaN,,NaN,NaN,19791,NaN,2020-01-20,0,1,1.0,2.0,2.0,2.0,2.0,2.0,0.0238,2,1,0.5000,0.018517,2.0,2.0,2.0,2.0,2.0,2.0,0.0238,2,1,0.5000,0.018517,5.0,2.0,2.0,2.0,1.0,2.0,0.0213,1,1,0.5000,0.001458,5.0,2.0,2.0,2.0,1.0,2.0,0.0213,1,1,0.5000,0.001458,1.0,2.0,1.0,2.0
2,4003,1000840,2020-01-29 18:01:59,SMART_BID_CONSERVATIVE,0,DEFAULT,0.0,INVENTORY_FEED|INVENTORY_HOTSOON_FEED|INVENTOR...,AD,FLOW_CONTROL_MODE_FAST,1110000000000000000011111111111111111111111111...,0,GENDER_MALE,11|12|13|14|15|21|22|23|31|32|33|34|35|36|37|4...,1,[],,,,NONE,CURRENT,,NONE,NONE,UNLIMITED,AGE_BETWEEN_24_30|AGE_BETWEEN_31_40|AGE_BETWEE...,,NaN,,NaN,NaN,19636,19,2020-01-29,1,1,10.0,10.0,10.0,10.0,10.0,10.0,0.0370,2,1,0.6970,0.275751,10.0,10.0,10.0,10.0,9.0,10.0,0.0664,2,1,0.7250,0.512874,8.0,10.0,10.0,10.0,3.0,6.0,0.0159,1,1,0.7143,0.356443,8.0,9.0,9.0,10.0,3.0,6.0,0.0201,1,1,0.7143,0.385200,1.0,3.0,1.0,122.0
3,4003,1000840,2020-01-29 18:02:00,SMART_BID_CONSERVATIVE,0,DEFAULT,0.0,INVENTORY_FEED|INVENTORY_HOTSOON_FEED|INVENTOR...,AD,FLOW_CONTROL_MODE_FAST,1110000000000000000000000000000000000111111111...,0,GENDER_MALE,11|12|13|14|15|21|22|23|31|32|33|34|35|36|37|4...,1,[],,,,NONE,CURRENT,,NONE,NONE,UNLIMITED,AGE_BETWEEN_24_30|AGE_BETWEEN_31_40|AGE_BETWEE...,,NaN,,NaN,NaN,19854,NaN,2020-01-29,0,1,10.0,10.0,10.0,10.0,9.0,9.0,0.0306,2,1,0.6774,0.312184,10.0,10.0,10.0,10.0,6.0,8.0,0.0568,2,1,0.7097,0.521922,8.0,10.0,10.0,10.0,3.0,6.0,0.0159,1,1,0.7143,0.356443,8.0,9.0,9.0,10.0,3.0,6.0,0.0201,1,1,0.7143,0.385200,1.0,4.0,1.0,24.0
4,4003,1000840,2020-01-30 14:32:38,SMART_BID_CONSERVATIVE,0,DEFAULT,0.0,INVENTORY_FEED|INVENTORY_HOTSOON_FEED|INVENTOR...,AD,FLOW_CONTROL_MODE_FAST,1111111111111111111111111111111111111111111111...,0,GENDER_MALE,11|12|13|14|15|21|22|23|31|32|33|34|35|36|37|4...,1,

In [39]:
df['label_ids'] = df['label_ids'].astype(str)
df['label_ids'] = df['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
df['label_ids'] = df['label_ids'].apply(lambda x: x.lstrip(','))

In [40]:
sparse_features = ["ad_account_id", "game_id",
                    "smart_bid_type", "hide_if_exists", "delivery_range", "adjust_cpa","hide_if_converted",
                   "flow_control_mode","schedule_time", "auto_extend_enabled",'gender',"platform","launch_price",
                  "android_osv","location_type","retargeting_type","ios_osv","interest_action_mode","action_days",
                   "deep_bid_type","image_id","label_ids"]
dense_features = ['im_amount_3',
       'im_create_role_num_3', 'im_new_role_money_3', 'im_pay_role_user_num_3',
       'im_source_num_3', 'im_valid_source_num_3', 'im_pay_rate_3',
       'im_create_role_cost_3', 'im_create_role_pay_cost_3',
       'im_valid_source_rate_3', 'image_create_role_roi_3', 'im_amount_5',
       'im_create_role_num_5', 'im_new_role_money_5', 'im_pay_role_user_num_5',
       'im_source_num_5', 'im_valid_source_num_5', 'im_pay_rate_5',
       'im_create_role_cost_5', 'im_create_role_pay_cost_5',
       'im_valid_source_rate_5', 'image_create_role_roi_5', 'ad_amount_3',
       'ad_create_role_num_3', 'ad_new_role_money_3', 'ad_pay_role_user_num_3',
       'ad_source_num_3', 'ad_valid_source_num_3', 'ad_pay_rate_3',
       'ad_create_role_cost_3', 'ad_create_role_pay_cost_3',
       'ad_valid_source_rate_3', 'ad_create_role_roi_3', 'ad_amount_5',
       'ad_create_role_num_5', 'ad_new_role_money_5', 'ad_pay_role_user_num_5',
       'ad_source_num_5', 'ad_valid_source_num_5', 'ad_pay_rate_5',
       'ad_create_role_cost_5', 'ad_create_role_pay_cost_5',
       'ad_valid_source_rate_5', 'ad_create_role_roi_5', 'ad_im_sort_id',
       'ad_game_sort_id', 'im_ad_sort_id', 'image_rank']
target = ['label']

In [41]:
df[dense_features] = df[dense_features].astype(np.float)
df[sparse_features] = df[sparse_features].fillna('-1')
df[dense_features] = df[dense_features].fillna(0)
df[sparse_features] = df[sparse_features].astype(str)

In [42]:
for feat in sparse_features:
    lbe = LabelEncoder()
    df[feat] = lbe.fit_transform(df[feat])
mms = MinMaxScaler(feature_range=(0, 1))
df[dense_features] = mms.fit_transform(df[dense_features])

In [43]:
train_data = df[df['train_label']==1]
test_data = df[df['train_label']==0]
train_data.drop(['train_label','create_time'],axis=1,inplace=True)
test_data.drop(['train_label','create_time'],axis=1,inplace=True)

C:\Users\Administrator\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [384]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=train_data[feat].max() + 1,embedding_dim=4)
                           for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                          for feat in dense_features]

In [137]:
def split(x):
    key_ans = x.split('|')
    for key in key_ans:
        if key not in key2index:
            # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], key_ans))

In [386]:
cols = ['inventory_type','age','city','retargeting_tags_include','retargeting_tags_exclude','ac','interest_categories',
       'action_scene','action_categories']
varlen_feature_columns = []
list_dict = {}
for col in cols:
    key2index = {}
    var_list = list(map(split, train_data[col].values))
    var_length = np.array(list(map(len, var_list)))
    max_len = max(var_length)
    var_list = pad_sequences(var_list, maxlen=max_len, padding='post', )
    list_dict[col] = var_list
    varlen_feature_columns += [VarLenSparseFeat(SparseFeat(col,vocabulary_size= len(
        key2index) + 1,embedding_dim=4), maxlen=max_len, combiner='mean',weight_name=None)]

In [312]:
linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

In [313]:
feature_names = get_feature_names(linear_feature_columns+dnn_feature_columns)

In [315]:
model_input = {name:train_data[name] for name in feature_names}#
for col in cols:
    model_input[col] = list_dict[col]
#     model_input[col] =  np.random.randn(df.shape[0],max_len,1)

In [323]:
model = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary',dnn_dropout=0.2,dnn_use_bn=True)

model.compile("adam", "binary_crossentropy", metrics=['AUC'], )
history = model.fit(model_input, df[target].values,
                        batch_size=256, epochs=20, verbose=2, validation_split=0.2, )

Epoch 1/20


C:\Users\Administrator\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


120/120 - 6s - loss: 0.4950 - auc: 0.5311 - val_loss: 0.3548 - val_auc: 0.4987
Epoch 2/20
120/120 - 2s - loss: 0.3155 - auc: 0.5884 - val_loss: 0.3490 - val_auc: 0.5274
Epoch 3/20
120/120 - 2s - loss: 0.3107 - auc: 0.6167 - val_loss: 0.3470 - val_auc: 0.5585
Epoch 4/20
120/120 - 2s - loss: 0.3035 - auc: 0.6513 - val_loss: 0.3539 - val_auc: 0.5573
Epoch 5/20
120/120 - 2s - loss: 0.2935 - auc: 0.6946 - val_loss: 0.3569 - val_auc: 0.5528
Epoch 6/20
120/120 - 2s - loss: 0.2824 - auc: 0.7317 - val_loss: 0.3655 - val_auc: 0.5432
Epoch 7/20
120/120 - 2s - loss: 0.2759 - auc: 0.7527 - val_loss: 0.3751 - val_auc: 0.5371
Epoch 8/20
120/120 - 2s - loss: 0.2688 - auc: 0.7748 - val_loss: 0.3880 - val_auc: 0.5376
Epoch 9/20
120/120 - 2s - loss: 0.2635 - auc: 0.7891 - val_loss: 0.3803 - val_auc: 0.5404
Epoch 10/20
120/120 - 2s - loss: 0.2577 - auc: 0.8025 - val_loss: 0.3904 - val_auc: 0.5365
Epoch 11/20
120/120 - 2s - loss: 0.2533 - auc: 0.8123 - val_loss: 0.4017 - val_auc: 0.5343
Epoch 12/20
120/120